In [66]:
import os

# Helper functions

In [69]:
def read_file(file_path):
    """
    Nolasa faila saturu.

    :param file_path: Ceļš uz failu.
    :return: Faila saturs baitos.
    """
    with open(file_path, 'rb') as f:
        content = f.read()
    return content

def write_file(file_path, content):
    """
    Ieraksta saturu failā.

    :param file_path: Ceļš uz failu.
    :param content: Saturs, ko rakstīt (baiti).
    """
    with open(file_path, 'wb') as f:
        f.write(content)

def pad_data(data):
    """
    Aizpilda datus, lai to garums būtu daudzkārtīgs 8 baitiem (64 biti).

    :param data: Dati, kas jāaizpilda (baiti).
    :return: Aizpildīti dati.
    """
    padding_len = 8 - (len(data) % 8)
    padding = bytes([padding_len] * padding_len)
    return data + padding

def unpad_data(data):
    """
    Noņem aizpildījumu no datiem.

    :param data: Aizpildīti dati (baiti).
    :return: Oriģinālie dati bez aizpildījuma.
    """
    padding_len = data[-1]
    return data[:-padding_len]

def bytearray_to_bits(data):
    """
    Konvertē baitu masīvu uz bitu sarakstu.

    :param data: Baitu masīvs vai baiti.
    :return: Bitu saraksts (kā virknes).
    """
    bits = []
    for byte in data:
        bits.extend(list(format(byte, '08b')))
    return bits

def bits_to_bytearray(bits):
    """
    Konvertē bitu sarakstu uz baitu masīvu.

    :param bits: Bitu saraksts (kā virknes).
    :return: Baitu masīvs.
    """
    bytes_list = []
    for i in range(0, len(bits), 8):
        byte = bits[i:i+8]
        bytes_list.append(int(''.join(byte), 2))
    return bytes(bytes_list)

# Initial permutation

In [72]:
# Sākotnējā permutācijas tabula
IP = [
    58, 50, 42, 34, 26, 18, 10, 2,
    60, 52, 44, 36, 28, 20, 12, 4,
    62, 54, 46, 38, 30, 22, 14, 6,
    64, 56, 48, 40, 32, 24, 16, 8,
    57, 49, 41, 33, 25, 17, 9, 1,
    59, 51, 43, 35, 27, 19, 11, 3,
    61, 53, 45, 37, 29, 21, 13, 5,
    63, 55, 47, 39, 31, 23, 15, 7
]

def initial_permutation(data):
    """
    Sākotnējā permutācija 64 bitu datu blokam

    Piemēram, pirmais elements 58 norāda, ka pirmajā pozīcijā jaunajā sarakstā būs bits, kas iepriekš atradās 58. pozīcijā.
    
    """
    return [data[i - 1] for i in IP] #pārkārtojam datus pēc IP tabulas


# Final permuation

In [75]:
# Galīgā permutācijas tabula
FP = [
    40, 8, 48, 16, 56, 24, 64, 32,
    39, 7, 47, 15, 55, 23, 63, 31,
    38, 6, 46, 14, 54, 22, 62, 30,
    37, 5, 45, 13, 53, 21, 61, 29,
    36, 4, 44, 12, 52, 20, 60, 28,
    35, 3, 43, 11, 51, 19, 59, 27,
    34, 2, 42, 10, 50, 18, 58, 26,
    33, 1, 41, 9, 49, 17, 57, 25
]

def final_permutation(data):
    # Galīgā permutācija 64 bitu blokam.
    return [data[i - 1] for i in FP]


# Expansion Function (E-Box)

In [78]:
# Paplašināšanas tabula (E)
E = [
    32, 1, 2, 3, 4, 5,
    4, 5, 6, 7, 8, 9,
    8, 9, 10, 11, 12, 13,
    12, 13, 14, 15, 16, 17,
    16, 17, 18, 19, 20, 21,
    20, 21, 22, 23, 24, 25,
    24, 25, 26, 27, 28, 29,
    28, 29, 30, 31, 32, 1
]

def expand(data):
    """
    Paplašina 32 bitu pusi-bloku līdz 48 bitiem, izmantojot E tabulu.

    :param data: Bitu saraksts (kā virkne) ar garumu 32.
    :return: Paplašināts bitu saraksts ar garumu 48.
    """
    return [data[i - 1] for i in E]

# Permutation Function (P-Box)

In [81]:
# Permutācijas tabula (P)
P = [
    16, 7, 20, 21,
    29, 12, 28, 17,
    1, 15, 23, 26,
    5, 18, 31, 10,
    2, 8, 24, 14,
    32, 27, 3, 9,
    19, 13, 30, 6,
    22, 11, 4, 25
]

def permute(data):
    """
    Permutē bitus, izmantojot P tabulu.

    :param data: Bitu saraksts (kā virkne) ar garumu 32.
    :return: Permutēts bitu saraksts.
    """
    return [data[i - 1] for i in P]


# S-boxes

In [84]:
S_boxes = {
    1: [
        [14, 4, 13, 1, 2, 15, 11, 8, 3, 10, 6, 12, 5, 9, 0, 7],
        [0, 15, 7, 4, 14, 2, 13, 1, 10, 6, 12, 11, 9, 5, 3, 8],
        [4, 1, 14, 8, 13, 6, 2, 11, 15, 12, 9, 7, 3, 10, 5, 0],
        [15, 12, 8, 2, 4, 9, 1, 7, 5, 11, 3, 14, 10, 0, 6, 13]
    ],
    2: [
        [15, 1, 8, 14, 6, 11, 3, 4, 9, 7, 2, 13, 12, 0, 5, 10],
        [3, 13, 4, 7, 15, 2, 8, 14, 12, 0, 1, 10, 6, 9, 11, 5],
        [0, 14, 7, 11, 10, 4, 13, 1, 5, 8, 12, 6, 9, 3, 2, 15],
        [13, 8, 10, 1, 3, 15, 4, 2, 11, 6, 7, 12, 0, 5, 14, 9]
    ],
    3: [
        [10, 0, 9, 14, 6, 3, 15, 5, 1, 13, 12, 7, 11, 4, 2, 8],
        [13, 7, 0, 9, 3, 4, 6, 10, 2, 8, 5, 14, 12, 11, 15, 1],
        [13, 6, 4, 9, 8, 15, 3, 0, 11, 1, 2, 12, 5, 10, 14, 7],
        [1, 10, 13, 0, 6, 9, 8, 7, 4, 15, 14, 3, 11, 5, 2, 12]
    ],
    4: [
        [7, 13, 14, 3, 0, 6, 9, 10, 1, 2, 8, 5, 11, 12, 4, 15],
        [13, 8, 11, 5, 6, 15, 0, 3, 4, 7, 2, 12, 1, 10, 14, 9],
        [10, 6, 9, 0, 12, 11, 7, 13, 15, 1, 3, 14, 5, 2, 8, 4],
        [3, 15, 0, 6, 10, 1, 13, 8, 9, 4, 5, 11, 12, 7, 2, 14]
    ],
    5: [
        [2, 12, 4, 1, 7, 10, 11, 6, 8, 5, 3, 15, 13, 0, 14, 9],
        [14, 11, 2, 12, 4, 7, 13, 1, 5, 0, 15, 10, 3, 9, 8, 6],
        [4, 2, 1, 11, 10, 13, 7, 8, 15, 9, 12, 5, 6, 3, 0, 14],
        [11, 8, 12, 7, 1, 14, 2, 13, 6, 15, 0, 9, 10, 4, 5, 3]
    ],
    6: [
        [12, 1, 10, 15, 9, 2, 6, 8, 0, 13, 3, 4, 14, 7, 5, 11],
        [10, 15, 4, 2, 7, 12, 9, 5, 6, 1, 13, 14, 0, 11, 3, 8],
        [9, 14, 15, 5, 2, 8, 12, 3, 7, 0, 4, 10, 1, 13, 11, 6],
        [4, 3, 2, 12, 9, 5, 15, 10, 11, 14, 1, 7, 6, 0, 8, 13]
    ],
    7: [
        [4, 11, 2, 14, 15, 0, 8, 13, 3, 12, 9, 7, 5, 10, 6, 1],
        [13, 0, 11, 7, 4, 9, 1, 10, 14, 3, 5, 12, 2, 15, 8, 6],
        [1, 4, 11, 13, 12, 3, 7, 14, 10, 15, 6, 8, 0, 5, 9, 2],
        [6, 11, 13, 8, 1, 4, 10, 7, 9, 5, 0, 15, 14, 2, 3, 12]
    ],
    8: [
        [13, 2, 8, 4, 6, 15, 11, 1, 10, 9, 3, 14, 5, 0, 12, 7],
        [1, 15, 13, 8, 10, 3, 7, 4, 12, 5, 6, 11, 0, 14, 9, 2],
        [7, 11, 4, 1, 9, 12, 14, 2, 0, 6, 10, 13, 15, 3, 5, 8],
        [2, 1, 14, 7, 4, 10, 8, 13, 15, 12, 9, 0, 3, 5, 6, 11]
    ]
}

def sbox_substitution(data):
    """
    Piemēro S-box aizvietošanas datiem.

    :param data: Bitu saraksts (kā virkne) ar garumu 48.
    :return: Aizvietots bitu saraksts ar garumu 32..
    """
    output = []
    for i in range(8):  # Ir 8 S-box
        block = data[i*6:(i+1)*6]  # ņem 6 bitus no katra S-box
        # Nosaka rindu un kolonnu S-box
        row = int(block[0] + block[5], 2)  # Pirmais un pēdējais bits veido rindu
        col = int(''.join(block[1:5]), 2)  # Vidējie četri biti veido kolonnu
        val = S_boxes[i+1][row][col]  # Iegūst S-box vērtību
        bin_val = format(val, '04b')  # Kovertē vērtību uz 4 bitu bināru virkni
        output.extend(list(bin_val))
    return output

# Key Schedule Functions

In [87]:
# Permutācijas Izvēle 1 (PC-1) tabula
PC1 = [
    57, 49, 41, 33, 25, 17, 9,
    1, 58, 50, 42, 34, 26, 18,
    10, 2, 59, 51, 43, 35, 27,
    19, 11, 3, 60, 52, 44, 36,

    63, 55, 47, 39, 31, 23, 15,
    7, 62, 54, 46, 38, 30, 22,
    14, 6, 61, 53, 45, 37, 29,
    21, 13, 5, 28, 20, 12, 4
]

# Permutācijas Izvēle 2 (PC-2) tabula
PC2 = [
    14, 17, 11, 24, 1, 5,
    3, 28, 15, 6, 21, 10,
    23, 19, 12, 4, 26, 8,
    16, 7, 27, 20, 13, 2,

    41, 52, 31, 37, 47, 55,
    30, 40, 51, 45, 33, 48,
    44, 49, 39, 56, 34, 53,
    46, 42, 50, 36, 29, 32
]

# Kreisā nobīde grafiks
SHIFT_SCHEDULE = [
    1, 1, 2, 2,
    2, 2, 2, 2,
    1, 2, 2, 2,
    2, 2, 2, 1
]

def left_shift(bits, n):
    """
    Veic kreiso apgriezienu nobīdi par n pozīcijām.

    :param bits: Bitu saraksts.
    :param n: Nobīdes skaits.
    :return: Bitu saraksts pēc nobīdes.
    """
    return bits[n:] + bits[:n]

def generate_keys(key):
    """
    Ģenerē 16 apakšatslēgas 16 kārtām.

    :param key: Bitu saraksts (kā virkne) ar garumu 64.
    :return: 16 apakšatslēgu saraksts, katra ar garumu 48 biti.
    """
    # Piemēro PC-1 atslēgai, lai iegūtu 56 bitu atslēgu (atmet paritātes bitus)
    permuted_key = [key[i - 1] for i in PC1]
    # Sadaliet atslēgu kreisajās un labajās pusēs
    left, right = permuted_key[:28], permuted_key[28:]
    keys = []
    for shift in SHIFT_SCHEDULE:
        # Kreisā nobīde abās pusēs
        left = left_shift(left, shift)
        right = left_shift(right, shift)
        # Apvienojiet pusēs un pielietojiet PC-2, lai iegūtu apakšatslēgu
        combined = left + right
        subkey = [combined[i - 1] for i in PC2]
        keys.append(subkey)
    return keys

# Feistel Function and DES Round Function

In [90]:
def feistel_function(right, subkey):
    """
    Piemēro Feisteļa funkciju datu labajai pusei.

    :param right: Labā puse datiem (kā bitu saraksts).
    :param subkey: Apakšatslēga, kas tiek izmantota šajā kārtā (kā bitu saraksts).
    :return: Rezultāts pēc Feisteļa funkcijas (kā bitu saraksts).
    """
    # 1. solis: Paplašināšana
    expanded_right = expand(right)
    # 2. solis: Atslēgas sajaukšana (XOR ar apakšatslēgu)
    xor_result = [str(int(b1) ^ int(b2)) for b1, b2 in zip(expanded_right, subkey)]
    # 3. solis: Aizvietošana, izmantojot S-kastes
    sbox_result = sbox_substitution(xor_result)
    # 4. solis: Permutācija
    permuted_result = permute(sbox_result)
    return permuted_result

def des_round(left, right, subkey, round_num):
    """
    Veic vienu DES algoritma kārtu.

    :param left: Kreisā puse datiem (kā bitu saraksts).
    :param right: Labā puse datiem (kā bitu saraksts).
    :param subkey: Apakšatslēga, kas tiek izmantota šajā kārtā (kā bitu saraksts).
    :param round_num: Kārtas numurs.
    :return: Jaunā kreisā un labā puse pēc šī solis.
    """
    # Piemēro Feisteļa funkciju labajai pusei
    f_output = feistel_function(right, subkey)
    
    # XOR rezultāts ar kreiso pusi
    new_right = [str(int(b1) ^ int(b2)) for b1, b2 in zip(left, f_output)]
    # Jaunā kreisā puse ir iepriekšējā labā puse
    new_left = right
    return new_left, new_right

# DES Encryption and Decryption Functions

In [93]:
def des_encrypt(block, keys):
    """
    Šifrē 64 bitu bloku, izmantojot DES.

    :param block: 64 bitu bloks (kā bitu saraksts).
    :param keys: 16 apakšatslēgas (katra ar garumu 48 biti).
    :return: Šifrēts teksts (kā bitu saraksts).
    """
    # Piemēro sākotnējo permutāciju
    block = initial_permutation(block)
    # Sadaliet kreisajās un labajās pusēs
    left, right = block[:32], block[32:]
    # Veiciet 16 kārtas
    for i in range(16):
        left, right = des_round(left, right, keys[i], i+1)
    # Apvienojiet pusēs (pēc pēdējās kārtas kreisā un labā puse tiek apmainīta vietām)
    combined = right + left
    # Piemēro galīgo permutāciju
    cipher_text = final_permutation(combined)
    return cipher_text

def des_decrypt(block, keys):
    """
    Atšifrē 64 bitu bloku, izmantojot DES.

    :param block: 64 bitu bloks (kā bitu saraksts).
    :param keys: 16 apakšatslēgas (katra ar garumu 48 biti).
    :return: Atšifrēts teksts (kā bitu saraksts).
    """
    # Piemēro sākotnējo permutāciju (IP)
    block = initial_permutation(block)
    # Sadala kreisajās un labajās pusēs
    left, right = block[:32], block[32:]
    # Veic 16 kārtas (izmantojot apakšatslēgas apgrieztā secībā)
    for i in range(16):
        left, right = des_round(left, right, keys[15 - i], i+1)
    # Apvieno puses
    combined = right + left
    # Piemēro galīgo permutāciju (FP)
    plain_text = final_permutation(combined)
    return plain_text

# 3DES Functions

In [96]:
def triple_des_encrypt(block, key1, key2, key3):
    """
    Šifrē 64 bitu bloku, izmantojot Trīskāršo DES.
    """
    # Ģenerē apakšatslēgas katrai atslēgai
    keys1 = generate_keys(key1)
    keys2 = generate_keys(key2)
    keys3 = generate_keys(key3)
    
    # Pirmais šifrējums
    block = des_encrypt(block, keys1)
    # Dekriptēšana ar otro atslēgu
    block = des_decrypt(block, keys2)
    # Noslēguma šifrējums ar trešo atslēgu
    block = des_encrypt(block, keys3)
    
    return block

def triple_des_decrypt(block, key1, key2, key3):
    """
    Atšifrē 64 bitu bloku, izmantojot Trīskāršo DES.
    """
    # Ģenerē apakšatslēgas katrai atslēgai
    keys1 = generate_keys(key1)
    keys2 = generate_keys(key2)
    keys3 = generate_keys(key3)
    
    # Pirmā dekripcija ar trešo atslēgu
    block = des_decrypt(block, keys3)
    # Šifrēšana ar otro atslēgu
    block = des_encrypt(block, keys2)
    # Noslēguma dekripcija ar pirmo atslēgu
    block = des_decrypt(block, keys1)
    
    return block


# File Encryption and Decryption Functions

In [109]:
def encrypt_file(input_path, output_path, key1, key2, key3):
    """
    Šifrē failu, izmantojot Trīskāršo DES.
    """
    print(f"Šifrējam failu: {input_path}")
    
    # Nolasīt ievades failu
    data = read_file(input_path)
    print("Ievades fails nolasīts.")
    
    # Pievienot aizpildījumu (padding), lai nodrošinātu, ka datu garums ir 64 bitu (8 baiti) reizinājums
    data = pad_data(data)
    print("Datiem pievienots aizpildījums.")
    
    encrypted_data = b''  # Iniciē tukšu baitu virkni šifrētajiem datiem
    
    # Konvertēt atslēgas bitu sarakstos (katra atslēga tiek izmantota Trīskāršās DES algoritmā)
    key1_bits = bytearray_to_bits(key1)
    key2_bits = bytearray_to_bits(key2)
    key3_bits = bytearray_to_bits(key3)
    print("Atslēgas konvertētas bitu sarakstos.")
    
    # Apstrādā katru 8 baitu (64 bitu) bloku no faila datiem
    for i in range(0, len(data), 8):
        block = data[i:i+8]  # Iegūt nākamo 8 baitu bloku
        if i < 80:  # Izpildām print tikai pirmajās 10 iterācijās (10 bloki, katrs pa 8 baitiem)
            print(f"Šifrējam bloku {i//8 + 1}: {block}")
        block_bits = bytearray_to_bits(block)  # Konvertē bloku bitu sarakstā
        cipher_bits = triple_des_encrypt(block_bits, key1_bits, key2_bits, key3_bits)  # Šifrē ar Trīskāršo DES
        encrypted_block = bits_to_bytearray(cipher_bits)  # Konvertē šifrēto bloku atpakaļ baitu sarakstā
        encrypted_data += encrypted_block  # Pievieno šifrēto bloku kopējam šifrētajiem datiem
    
    # Ierakstīt šifrētos datus norādītajā izejas failā
    write_file(output_path, encrypted_data)
    print(f"Šifrētie dati ierakstīti failā: {output_path}")


def decrypt_file(input_path, output_path, key1, key2, key3):
    """
    Atšifrē failu, izmantojot Trīskāršo DES.
    """
    print(f"Atšifrējam failu: {input_path}")
    
    # Nolasa šifrēto failu no norādītā ceļa
    data = read_file(input_path)
    print("Šifrētais fails nolasīts.")
    
    decrypted_data = b''  # Iniciē tukšu baitu virkni atšifrētajiem datiem
    
    # Konvertēt atslēgas bitu sarakstos (katra atslēga tiks izmantota Trīskāršās DES dešifrēšanas algoritmā)
    key1_bits = bytearray_to_bits(key1)
    key2_bits = bytearray_to_bits(key2)
    key3_bits = bytearray_to_bits(key3)
    print("Atslēgas konvertētas bitu sarakstos.")
    
    # Apstrādā katru 8 baitu (64 bitu) bloku no šifrētajiem datiem
    for i in range(0, len(data), 8):
        block = data[i:i+8]  # Iegūt nākamo 8 baitu bloku
        if i < 80:  # Izpildām print tikai pirmajās 10 iterācijās (10 bloki, katrs pa 8 baitiem)
            print(f"Atšifrējam bloku: {block}")
        block_bits = bytearray_to_bits(block)  # Konvertē bloku bitu sarakstā
        plain_bits = triple_des_decrypt(block_bits, key1_bits, key2_bits, key3_bits)  # Atšifrē ar Trīskāršo DES
        decrypted_block = bits_to_bytearray(plain_bits)  # Konvertē atšifrēto bloku atpakaļ baitu sarakstā
        decrypted_data += decrypted_block  # Pievieno atšifrēto bloku kopējam atšifrētajiem datiem
    
    # Noņemt aizpildījumu (padding), kas tika pievienots šifrēšanas laikā
    decrypted_data = unpad_data(decrypted_data)
    print("Aizpildījums noņemts no atšifrētajiem datiem.")
    
    # Ierakstīt atšifrētos datus norādītajā izejas failā
    write_file(output_path, decrypted_data)
    print(f"Atšifrētie dati ierakstīti failā: {output_path}")


# Main Method

In [112]:
# Sample keys (must be 8 bytes each)
key1 = b'12345678'
key2 = b'23456789'
key3 = b'87654321'

# Ceļš uz failiem
input_pdf = 'test.pdf'  # Replace with your input PDF file name
encrypted_pdf = 'encrypted_sample.pdf'
decrypted_pdf = 'decrypted_sample.pdf'

# Šifrē PDF
print("Starting encryption...")
encrypt_file(input_pdf, encrypted_pdf, key1, key2, key3)
print(f"File '{input_pdf}' has been encrypted to '{encrypted_pdf}'.")

# Atšifrē PDF
print("Starting decryption...")
decrypt_file(encrypted_pdf, decrypted_pdf, key1, key2, key3)
print(f"File '{encrypted_pdf}' has been decrypted to '{decrypted_pdf}'.")


Starting encryption...
Šifrējam failu: test.pdf
Ievades fails nolasīts.
Datiem pievienots aizpildījums.
Atslēgas konvertētas bitu sarakstos.
Šifrējam bloku 1: b'%PDF-1.7'
Šifrējam bloku 2: b'\n\n4 0 ob'
Šifrējam bloku 3: b'j\n(Ident'
Šifrējam bloku 4: b'ity)\nend'
Šifrējam bloku 5: b'obj\n5 0 '
Šifrējam bloku 6: b'obj\n(Ado'
Šifrējam bloku 7: b'be)\nendo'
Šifrējam bloku 8: b'bj\n8 0 o'
Šifrējam bloku 9: b'bj\n<<\n/F'
Šifrējam bloku 10: b'ilter /F'
Šifrētie dati ierakstīti failā: encrypted_sample.pdf
File 'test.pdf' has been encrypted to 'encrypted_sample.pdf'.
Starting decryption...
Atšifrējam failu: encrypted_sample.pdf
Šifrētais fails nolasīts.
Atslēgas konvertētas bitu sarakstos.
Atšifrējam bloku: b'\x05\xd9\x99;\xa9\x0e\x16\x8f'
Atšifrējam bloku: b'\xaf[3B\x00\x04\xe7B'
Atšifrējam bloku: b'\xe2\x98\x00\xc3\xbdAnZ'
Atšifrējam bloku: b';tAp+\xce\x0f\xbc'
Atšifrējam bloku: b'\xbew\xbc\xf0\x8d \xa1\xd8'
Atšifrējam bloku: b'\xbd(\xef\xe9Z\xad\xfaO'
Atšifrējam bloku: b'\t\xd9\xaf\x9a\xe5